In [3]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-09-12 09:55 ajna         INFO     Configuração de log efetuada


2025-09-12 09:55:18,105 ajna         INFO     Configuração de log efetuada


In [22]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [65]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores
LISTA_REDEX = "('8931309', '8933204')"
FILTRO_LIKE_REDEX = "'89327%'"  # Redex locais específicos

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=3)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [67]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
"""

In [68]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""

In [69]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""

In [70]:
# SQL para saida_redex (alias c)
sql_saida_redex = f"""
SELECT 
    id, placa, numeroConteiner, codigoRecinto, dataHoraOcorrencia, direcao
FROM apirecintos_acessosveiculo
WHERE
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'S'
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""

In [71]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [72]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [73]:
df_saida_redex

,id,placa,numeroConteiner,codigoRecinto,dataHoraOcorrencia,direcao
0,12231188,DPC9996,,8932797,2025-09-09 10:39:27,S
1,12228905,JCQ6A11,,8932709,2025-09-09 10:40:16,S
2,12230766,BWP0G99,,8932796,2025-09-09 10:40:17,S
3,12228932,EPU9619,,8932709,2025-09-09 10:41:19,S
4,12229138,SPE6D90,,8932710,2025-09-09 10:41:20,S
...,...,...,...,...,...,...
3765,12297976,TJR2C42,,8932761,2025-09-11 23:58:04,S
3766,12297269,JBT5A66,,8932722,2025-09-11 23:58:09,S
3767,12298581,CZB0F90,,8932796,2025-09-11 23:58:36,S
3768,12298582,EJV2C48,,8932796,2025-09-11 23:58:51,S


In [74]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='left', on=['placa', 'numeroConteiner'], suffixes=('_E_Operador', '_S_REDEX'))

In [78]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [79]:
df_saidas_entradas

,id_E_Operador,placa,numeroConteiner,codigoRecinto_E_Operador,dataHoraOcorrencia_E_Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,id_S_REDEX,codigoRecinto_S_REDEX,dataHoraOcorrencia_S_REDEX,direcao_S_REDEX,transitTime
0,12217074,FEI3B16,EGHU8203814,8931318,2025-09-09 10:40:26,C,E,12014047000193,04596185557,MAURICIO NUNES,,,NaN,NaN,NaT,NaN,NaN
1,12217081,RHF4E57,,8931318,2025-09-09 10:48:57,C,E,19199348000188,10655443754,LEANDRO HENRIQUE DA SILVA,"31250916701716000156550250077237931525856491, ...",,NaN,NaN,NaT,NaN,NaN
2,12217084,BXF3791,,8931318,2025-09-09 11:06:22,C,E,50380680000194,43182387847,LUCAS FERREIRA DE LIMA,,,NaN,NaN,NaT,NaN,NaN
3,12217085,CSK3A11,EGSU3174039,8931318,2025-09-09 11:10:54,C,E,21425093000176,31798854830,LEANDRO NASCIMENTO LIMA,,,NaN,NaN,NaT,NaN,NaN
4,12217087,SRW4F37,,8931318,2025-09-09 11:23:35,C,E,13411081000164,01856659771,JORGE MARIO SILVA SANTOS,,,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21310,12296748,GIA6H99,,8931404,2025-09-11 23:52:05,C,E,61102778000287,,ROBERT GABRIEL COFFANI CRESPO,,,NaN,NaN,NaT,NaN,NaN
21311,12296752,MTN9C73,TGBU3928734,8931404,2025-09-11 23:52:14,C,E,43388833000184,41116258862,FABIO HENRIQUE DE SOUZA SANTOS,,,NaN,NaN,NaT,NaN,NaN
21312,12296751,CUA6E84,,8931404,2025-09-11 23:52:37,C,E,45050663000159,31455878898,ALEXSANDRO BARBOSA DA CRUZ,,,NaN,NaN,NaT,NaN,NaN
21313,12296755,MFO5H96,,8931404,2025-09-11 23:53:52,C,E,01040049000130,04796650601,NEILTON SANTANA DE SÃ?,,,NaN,NaN,NaT,NaN,NaN


In [80]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx')

In [ ]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placa', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 2) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placa', 'numeroConteiner'], right_on=['placa', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 2) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) &
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

NameError: name 'engine' is not defined